# بناء باستخدام نماذج Mistral

## المقدمة

ستغطي هذه الدرس:
- استكشاف نماذج Mistral المختلفة
- فهم حالات الاستخدام والسيناريوهات لكل نموذج
- عينات من الشيفرة توضح الميزات الفريدة لكل نموذج


## نماذج ميسترال

في هذا الدرس، سنستعرض ثلاثة نماذج مختلفة من ميسترال:
**ميسترال لارج**، **ميسترال سمول** و **ميسترال نيمو**.

كل هذه النماذج متاحة مجاناً على سوق النماذج في Github. الكود في هذا الدفتر سيستخدم هذه النماذج لتشغيل الشيفرة. إليك المزيد من التفاصيل حول استخدام نماذج Github لـ [النمذجة الأولية مع نماذج الذكاء الاصطناعي](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## ميسترال لارج 2 (2407)
ميسترال لارج 2 هو حالياً النموذج الرائد من ميسترال ومصمم للاستخدام المؤسسي.

هذا النموذج يُعد ترقية عن ميسترال لارج الأصلي من خلال تقديم:
- نافذة سياق أكبر - 128 ألف مقابل 32 ألف
- أداء أفضل في مهام الرياضيات والبرمجة - دقة متوسطة 76.9% مقابل 60.4%
- أداء متعدد اللغات محسّن - تشمل اللغات: الإنجليزية، الفرنسية، الألمانية، الإسبانية، الإيطالية، البرتغالية، الهولندية، الروسية، الصينية، اليابانية، الكورية، العربية، والهندية.

مع هذه الميزات، يتفوق ميسترال لارج في:
- *توليد المعرفة المعزز بالاسترجاع (RAG)* - بفضل نافذة السياق الأكبر
- *استدعاء الدوال* - هذا النموذج يدعم استدعاء الدوال بشكل أصلي، مما يسمح بالتكامل مع الأدوات وواجهات البرمجة الخارجية. يمكن إجراء هذه الاستدعاءات بشكل متوازي أو متسلسل واحداً تلو الآخر.
- *توليد الشيفرة البرمجية* - هذا النموذج يتفوق في توليد شيفرات بايثون، جافا، تايب سكريبت وC++.


### مثال RAG باستخدام Mistral Large 2


في هذا المثال، نستخدم Mistral Large 2 لتطبيق نمط RAG على مستند نصي. السؤال مكتوب بالكورية ويسأل عن أنشطة المؤلف قبل الجامعة.

يتم استخدام نموذج Cohere Embeddings لإنشاء تمثيلات مضمنة للمستند النصي وكذلك للسؤال. في هذا المثال، يتم استخدام حزمة faiss في بايثون كمخزن متجهات.

يتضمن الطلب المرسل إلى نموذج Mistral كل من الأسئلة والمقاطع المسترجعة التي تشبه السؤال. بعد ذلك، يقدم النموذج إجابة بلغة طبيعية.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## ميسترال سمول
ميسترال سمول هو نموذج آخر ضمن عائلة ميسترال من النماذج، ويأتي تحت فئة النماذج المميزة/المؤسساتية. كما يوحي الاسم، هذا النموذج هو نموذج لغوي صغير (SLM). من مزايا استخدام ميسترال سمول أنه:
- يوفر في التكاليف مقارنة بنماذج ميسترال الكبيرة مثل ميسترال لارج وNeMo - انخفاض في السعر بنسبة 80%
- زمن استجابة منخفض - أسرع في الرد مقارنة بالنماذج اللغوية الكبيرة من ميسترال
- مرن - يمكن نشره في بيئات مختلفة مع قيود أقل على الموارد المطلوبة.

ميسترال سمول مناسب لـ:
- المهام النصية مثل التلخيص، وتحليل المشاعر، والترجمة.
- التطبيقات التي تتطلب طلبات متكررة بسبب فعاليته من حيث التكلفة
- مهام البرمجة التي تتطلب استجابة سريعة مثل مراجعة الكود واقتراحات الكود


## مقارنة بين Mistral Small و Mistral Large

لإظهار الفروقات في زمن الاستجابة بين Mistral Small و Mistral Large، قم بتشغيل الخلايا أدناه.

يجب أن تلاحظ فرقاً في أوقات الاستجابة يتراوح بين 3 إلى 5 ثوانٍ. لاحظ أيضاً اختلاف طول الإجابات وأسلوبها عند استخدام نفس الطلب.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## ميسترال ني مو

مقارنة بالنموذجين الآخرين الذين تم مناقشتهما في هذا الدرس، ميسترال ني مو هو النموذج المجاني الوحيد الذي يحمل رخصة أباتشي 2.

يُعتبر هذا النموذج بمثابة تطوير للنموذج مفتوح المصدر السابق من ميسترال، ميسترال 7B.

بعض الميزات الأخرى لنموذج ني مو هي:

- *ترميز أكثر كفاءة:* يستخدم هذا النموذج أداة الترميز Tekken بدلاً من tiktoken الأكثر شيوعاً. هذا يمنحه أداءً أفضل عبر لغات وبرمجيات متعددة.

- *إمكانية التخصيص (Finetuning):* النموذج الأساسي متاح للتخصيص. هذا يوفر مرونة أكبر في الحالات التي قد تتطلب تخصيص النموذج.

- *استدعاء الدوال بشكل أصلي* - مثل Mistral Large، تم تدريب هذا النموذج على استدعاء الدوال. وهذا يجعله مميزاً كونه من أوائل النماذج مفتوحة المصدر التي تدعم هذه الميزة.


## ميسترال ني مو

مقارنة بالنموذجين الآخرين الذين تم مناقشتهما في هذا الدرس، ميسترال ني مو هو النموذج المجاني الوحيد الذي يحمل رخصة أباتشي 2.

يُعتبر هذا النموذج بمثابة تطوير للنموذج مفتوح المصدر السابق من ميسترال، ميسترال 7B.

بعض الميزات الأخرى لنموذج ني مو هي:

- *ترميز أكثر كفاءة:* يستخدم هذا النموذج أداة الترميز Tekken بدلاً من tiktoken الأكثر شيوعاً. هذا يسمح بأداء أفضل عبر لغات وبرمجيات أكثر.

- *إمكانية التخصيص (Finetuning):* النموذج الأساسي متاح للتخصيص. هذا يوفر مرونة أكبر في الحالات التي قد تتطلب تخصيص النموذج.

- *استدعاء الدوال بشكل أصلي* - مثل Mistral Large، تم تدريب هذا النموذج على استدعاء الدوال. وهذا يجعله مميزاً كونه من أوائل النماذج مفتوحة المصدر التي تدعم هذه الميزة.


### مقارنة أدوات تقسيم النصوص

في هذا المثال، سنستعرض كيف يتعامل Mistral NeMo مع تقسيم النصوص مقارنةً بـ Mistral Large.

كلا المثالين يستخدمان نفس الطلب، لكن يجب أن تلاحظ أن NeMo يعيد عددًا أقل من الرموز مقارنةً بـ Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## التعلم لا يتوقف هنا، واصل الرحلة

بعد الانتهاء من هذا الدرس، اطلع على [مجموعة تعلم الذكاء الاصطناعي التوليدي](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) لمواصلة تطوير معرفتك في الذكاء الاصطناعي التوليدي!



---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للمعلومات الهامة، يُنصح بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسير خاطئ ينشأ عن استخدام هذه الترجمة.
